In [11]:
%cd ~/REVIVAL2
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

/disk2/fli/REVIVAL2
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


From Chai docked structure
extract substrate and the rest
each --> pdbqt
--> vina score only

ie

`obabel PfTrpB-4bromo_0.cif -O PfTrpB-4bromo_0.pdbqt`

`vina --receptor receptor.pdbqt --ligand PfTrpB-4bromo_0.pdbqt --score_only --out output.pdbqt --log score.log`

```
import os
import subprocess

def get_docking_score(receptor_path, ligand_path):
    log_file = "vina_score.log"
    try:
        # Run Vina in score-only mode
        subprocess.run(
            [
                "vina",
                "--receptor", receptor_path,
                "--ligand", ligand_path,
                "--score_only",
                "--log", log_file
            ],
            check=True
        )
        
        # Parse the log file for affinity
        with open(log_file, "r") as log:
            for line in log:
                if "Affinity" in line:
                    score = float(line.split(":")[1].strip().split()[0])
                    return score
    except Exception as e:
        print(f"Error in scoring: {e}")
        return None

# Example usage
receptor = "receptor.pdbqt"
ligand = "PfTrpB-4bromo_0.pdbqt"
score = get_docking_score(receptor, ligand)
print(f"Docking Score: {score} kcal/mol")
```

In [7]:
def merge_pdbqt(input_file_path, output_file_path):
    """
    Merge multiple components in a PDBQT file into a single molecule under one ROOT.

    Args:
        input_file_path (str): Path to the input PDBQT file.
        output_file_path (str): Path to save the merged PDBQT file.
    """
    with open(input_file_path, "r") as infile, open(output_file_path, "w") as outfile:
        for line in infile:
            if line.startswith("ATOM"):
                outfile.write(line)
        
        outfile.write("TER\n")

    print(f"File with only ATOM lines saved to {output_file_path}")


# Example usage
input_file = "/disk2/fli/REVIVAL2/sandbox/files/WT_0/comb.pdbqt"  # Replace with your input file
output_file = "/disk2/fli/REVIVAL2/sandbox/files/WT_0/comb_cleaned.pdbqt"  # Replace with your output file
merge_pdbqt(input_file, output_file)


File with only ATOM lines saved to /disk2/fli/REVIVAL2/sandbox/files/WT_0/comb_cleaned.pdbqt


In [13]:
def extract_atom_lines_from_multiple_files(input_files, output_file_path):
    """
    Extract and save only lines starting with ATOM from multiple files, 
    and combine them into one output file with a final TER line.

    Args:
        input_files (list): List of input file paths to be processed.
        output_file_path (str): Path to save the combined output file.
    """
    with open(output_file_path, "w") as outfile:

        for input_file_path in input_files:
            with open(input_file_path, "r") as infile:
                for line in infile:
                    if line.startswith("ATOM"):
                        outfile.write(line)

        # Add the TER line to indicate termination
        outfile.write("TER\n")

    print(f"Combined ATOM lines saved to {output_file_path}")

folder = "/disk2/fli/REVIVAL2/sandbox/files/WT_0/"
# Example usage
input_files = [folder+"WT_0.pdbqt", folder+"Na.pdbqt", folder+"PLP-dependent_aminoacrylate.pdbqt"]  # Replace with your actual files
output_file = "combNclean.pdbqt"  # Replace with your desired output file
extract_atom_lines_from_multiple_files(input_files, output_file)


Combined ATOM lines saved to combNclean.pdbqt


In [4]:
import sys
import os
import numpy
import openmm
print(f"Python: {sys.version}")
print(f"NumPy: {numpy.__version__}")
print(f"OpenMM: {openmm.version.version}")
print(f"Environment: {os.environ}")

Python: 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) 
[GCC 9.4.0]
NumPy: 2.0.2
OpenMM: 7.7.0.dev-130124a
Environment: environ({'USER': 'fli', 'SSH_CLIENT': '131.215.248.58 49974 22', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '2', 'MOTD_SHOWN': 'pam', 'HOME': '/disk2/fli', 'SSL_CERT_FILE': '/usr/lib/ssl/certs/ca-certificates.crt', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1002/bus', 'LOGNAME': 'fli', '_': '/disk2/fli/miniconda3/envs/vina/bin/python', 'XDG_SESSION_CLASS': 'user', 'XDG_SESSION_ID': '8448', 'VSCODE_CLI_REQUIRE_TOKEN': 'e2fff225-9583-46c0-8e69-bfcf24b4c518', 'PATH': '/disk2/fli/miniconda3/envs/vina/bin:/disk2/fli/.vscode-server/cli/servers/Stable-fabdb6a30b49f79a7aba0f2ad9df9b399473380f/server/bin/remote-cli:/disk2/fli/.local/bin:/disk2/fli/orca_6_0_1:/disk2/fli/miniconda3/envs/vina/bin:/disk2/fli/miniconda3/condabin:/disk2/fli/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin', 'VSCODE_AGENT

In [14]:
import os
import psutil

total_cpus = os.cpu_count()
available_cpus = len(psutil.Process().cpu_affinity())

print(f"Total CPUs: {total_cpus}")
print(f"Available CPUs: {available_cpus}")


Total CPUs: 384
Available CPUs: 384


In [2]:
from REVIVAL.util import get_chain_structure

/disk2/fli/miniconda3/envs/vina/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [22]:
get_chain_structure(
    input_file_path="/disk2/fli/REVIVAL2/sandbox/files/WT_0/WT_0.cif",
    output_file_path="/disk2/fli/REVIVAL2/sandbox/files/WT_0/WT_0_A.pdb",
    chain_id="A",
)

Chains A have been saved to /disk2/fli/REVIVAL2/sandbox/files/WT_0/WT_0_A.pdb


In [23]:
get_chain_structure(
    input_file_path="/disk2/fli/REVIVAL2/sandbox/files/WT_0/WT_0.cif",
    output_file_path="/disk2/fli/REVIVAL2/sandbox/files/WT_0/WT_0_B.pdb",
    chain_id="B",
)

Chains B have been saved to /disk2/fli/REVIVAL2/sandbox/files/WT_0/WT_0_B.pdb


In [9]:
from rdkit import Chem
from rdkit.Chem import AllChem
from Bio.PDB import PDBParser, PDBIO, Structure
import numpy as np
import os

def extract_molecule_and_split_pdb(pdb_file_path: str, smiles: str):
    """
    Extract a small molecule from a PDB file based on a SMILES string
    and return both the small molecule and the rest of the structure.

    Args:
        pdb_file_path (str): Path to the input PDB file.
        smiles (str): SMILES string of the molecule to extract.

    Returns:
        tuple: Two Bio.PDB.Structure objects: one for the small molecule
               and one for the rest of the original structure.
    """
    # Parse the input PDB file
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file_path)

    # Convert SMILES to a molecule and generate 3D structure
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError(f"Invalid SMILES string: {smiles}")
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())

    # Extract coordinates of the small molecule from RDKit
    ref_coords = np.array(mol.GetConformer().GetPositions())

    small_molecule_atoms = []
    remaining_atoms = []

    # Iterate through the PDB structure and classify atoms
    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    atom_coords = np.array(atom.coord)
                    # Check if the atom matches with the small molecule
                    if np.any(np.linalg.norm(ref_coords - atom_coords, axis=1) < 1.0):
                        small_molecule_atoms.append(atom)
                    else:
                        remaining_atoms.append(atom)

    # Create separate structures for the small molecule and the rest
    small_molecule_structure = Structure.Structure("small_molecule")
    remaining_structure = Structure.Structure("remaining_structure")

    small_model = small_molecule_structure.add(model.copy())
    remaining_model = remaining_structure.add(model.copy())

    for chain in model:
        small_chain = chain.copy()
        remaining_chain = chain.copy()
        small_chain.detach_children()
        remaining_chain.detach_children()

        for residue in chain:
            small_residue = residue.copy()
            remaining_residue = residue.copy()
            small_residue.detach_children()
            remaining_residue.detach_children()

            for atom in residue:
                if atom in small_molecule_atoms:
                    small_residue.add(atom)
                else:
                    remaining_residue.add(atom)

            if len(small_residue):
                small_chain.add(small_residue)
            if len(remaining_residue):
                remaining_chain.add(remaining_residue)

        if len(small_chain):
            small_model.add(small_chain)
        if len(remaining_chain):
            remaining_model.add(remaining_chain)

    return small_molecule_structure, remaining_structure


In [10]:
from Bio.PDB import PDBParser, PDBIO, MMCIFParser
from rdkit import Chem
from rdkit.Chem import AllChem
import os

def extract_molecule_with_remainder(input_file, smile_string, small_molecule_pdb, rest_structure_pdb):
    """
    Extracts a small molecule matching the given SMILES string from a PDB or CIF file
    and saves both the small molecule and the remaining structure into separate files.

    Args:
        input_file (str): Path to the input PDB or CIF file.
        smile_string (str): SMILES string of the small molecule to extract.
        small_molecule_pdb (str): Path to save the extracted small molecule PDB file.
        rest_structure_pdb (str): Path to save the remaining structure PDB file.
    """
    # Determine the parser based on file extension
    if input_file.endswith(".cif"):
        parser = MMCIFParser()
    elif input_file.endswith(".pdb"):
        parser = PDBParser()
    else:
        raise ValueError("Unsupported file format. Only .pdb and .cif are supported.")

    # Parse the structure
    structure = parser.get_structure("input_structure", input_file)

    # Generate RDKit molecule from SMILES
    smiles_mol = Chem.MolFromSmiles(smile_string)
    if not smiles_mol:
        raise ValueError("Invalid SMILES string.")

    # Add hydrogens to the RDKit molecule
    # smiles_mol = Chem.AddHs(smiles_mol)
    AllChem.EmbedMolecule(smiles_mol)

    # Containers for small molecule atoms and remaining structure atoms
    small_molecule_atoms = []
    remaining_atoms = []

    # Iterate over all atoms to classify them
    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    atom_coords = atom.get_coord()
                    # Check if atom matches the small molecule using RDKit
                    atom_mol = Chem.MolFromSmiles(atom.element)
                    if atom_mol and Chem.rdMolAlign.GetBestRMS(smiles_mol, atom_mol) < 1.0:
                        small_molecule_atoms.append(atom)
                    else:
                        remaining_atoms.append(atom)

    # Write the small molecule atoms to the PDB file
    if small_molecule_atoms:
        io = PDBIO()
        io.set_structure(small_molecule_atoms)
        io.save(small_molecule_pdb)

    # Write the remaining structure atoms to the PDB file
    if remaining_atoms:
        io = PDBIO()
        io.set_structure(remaining_atoms)
        io.save(rest_structure_pdb)

    print(f"Small molecule saved to {small_molecule_pdb}")
    print(f"Remaining structure saved to {rest_structure_pdb}")

In [18]:
import os
from Bio.PDB import PDBParser, MMCIFParser, PDBIO, MMCIFIO

def extract_substrate_by_criteria(input_file_path, output_substrate_path, output_rest_path, criteria):
    """
    Extract a substrate from a PDB or CIF file based on chain, resn, and atom name.

    Args:
        input_file_path (str): Path to the input PDB or CIF file.
        output_substrate_path (str): Path to save the extracted substrate structure.
        output_rest_path (str): Path to save the rest of the structure.
        criteria (list of tuples): List of tuples specifying the selection criteria.
                                   Each tuple should contain (chain_id, resn, name).
    """
    is_cif = input_file_path.endswith(".cif")

    if is_cif:
        parser = MMCIFParser(QUIET=True)
        structure = parser.get_structure("protein", input_file_path)
    else:
        parser = PDBParser(QUIET=True)
        structure = parser.get_structure("protein", input_file_path)

    substrate_atoms = []
    rest_atoms = []

    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    match = any(
                        chain.id == chain_id and residue.resname == resn and atom.name == name
                        for chain_id, resn, name in criteria
                    )
                    if match:
                        substrate_atoms.append(atom)
                    else:
                        rest_atoms.append(atom)

    # Create structures for substrate and rest
    substrate_structure = structure.copy()
    rest_structure = structure.copy()

    for model in substrate_structure:
        for chain in model:
            for residue in list(chain):
                for atom in list(residue):
                    if atom not in substrate_atoms:
                        residue.detach_child(atom.name)
    for model in rest_structure:
        for chain in model:
            for residue in list(chain):
                for atom in list(residue):
                    if atom in substrate_atoms:
                        residue.detach_child(atom.name)

    # Save structures
    io = PDBIO()
    io.set_structure(substrate_structure)
    io.save(output_substrate_path)
    io.set_structure(rest_structure)
    io.save(output_rest_path)

    print(f"Substrate saved to {output_substrate_path}")
    print(f"Rest of the structure saved to {output_rest_path}")


In [19]:
extract_substrate_by_criteria(
    input_file_path="/disk2/fli/docko/tests/fzl/revival2/PfTrpB-4bromo/PfTrpB-4bromo_0.cif", 
    output_substrate_path="/disk2/fli/REVIVAL2/sandbox/files/PfTrpB-4bromo_0_molecule.pdb", 
    output_rest_path="/disk2/fli/REVIVAL2/sandbox/files/PfTrpB-4bromo_0_remainder.pdb",
    criteria=[
        ('B', 'LIG', 'C_1'),
        ('B', 'LIG', 'C_2'),
        ('B', 'LIG', 'C_3'),
        ('B', 'LIG', 'C_4'),
        ('B', 'LIG', 'C_5'),
        ('B', 'LIG', 'C_6'),
        ('B', 'LIG', 'C_7'),
        ('B', 'LIG', 'C_8'),
        ('B', 'LIG', 'N_1'),
        ('B', 'LIG', 'BR_1')
    ])

Substrate saved to /disk2/fli/REVIVAL2/sandbox/files/PfTrpB-4bromo_0_molecule.pdb
Rest of the structure saved to /disk2/fli/REVIVAL2/sandbox/files/PfTrpB-4bromo_0_remainder.pdb


In [24]:
from REVIVAL.util import convert_cif_to_pdb

In [25]:
convert_cif_to_pdb(
    cif_file="/disk2/fli/REVIVAL2/sandbox/files/pgb/jlk_ee-test_ape_ygpw_aze_model.cif",
    pdb_file="/disk2/fli/REVIVAL2/sandbox/files/pgb/jlk_ee-test_ape_ygpw_aze_model.pdb",
    ifsave = True
)

Converted /disk2/fli/REVIVAL2/sandbox/files/pgb/jlk_ee-test_ape_ygpw_aze_model.cif to /disk2/fli/REVIVAL2/sandbox/files/pgb/jlk_ee-test_ape_ygpw_aze_model.pdb


In [8]:
from Bio.PDB import PDBParser, MMCIFParser
import numpy as np
import os

def calculate_chain_centroid(input_file, chain_ids):
    """
    Calculate the geometric center (centroid) of all atoms in the specified chain(s).

    Args:
        input_file (str): Path to the input PDB or CIF file.
        chain_ids (list of str): List of chain IDs to calculate the centroid for.

    Returns:
        tuple: The XYZ coordinates of the centroid.
    """
    # Determine the file type
    file_extension = os.path.splitext(input_file)[-1].lower()
    if file_extension == ".cif":
        parser = MMCIFParser(QUIET=True)
    elif file_extension == ".pdb":
        parser = PDBParser(QUIET=True)
    else:
        raise ValueError("Unsupported file format. Only PDB and CIF files are supported.")

    # Parse the structure
    structure = parser.get_structure("protein", input_file)

    coordinates = []
    chain_ids = [cid.upper() for cid in chain_ids]  # Ensure chain IDs are uppercase

    for model in structure:
        for chain in model:
            if chain.id.upper() in chain_ids:
                for residue in chain:
                    for atom in residue:
                        coordinates.append(atom.coord)

    # Calculate centroid
    if coordinates:
        centroid = np.mean(coordinates, axis=0)
        return np.array(centroid).flatten()
    else:
        raise ValueError(f"No atoms found for the specified chain(s): {chain_ids}")

# # Example usage
# input_file = "example.cif"  # Path to your PDB or CIF file
# chain_ids = ["A", "B"]      # Chain IDs to calculate the centroid for
# center = calculate_chain_centroid(input_file, chain_ids)
# print("Center of mass:", center)


In [9]:
calculate_chain_centroid("/disk2/fli/REVIVAL2/sandbox/files/WT_0/WT_0.cif", "B")

array([2.7593548 , 0.81696784, 4.430387  ], dtype=float32)